In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Resnet152


In [ ]:
from keras.applications.densenet import DenseNet201
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np
from keras.applications.resnet_v2 import ResNet152V2

# ResNet152 모델 로드 (최상위 층 제외)
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강 없이 이미지 크기만 조정하는 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 데이터셋 로드 및 설정
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = train_generator.samples // train_generator.batch_size
if train_generator.samples % train_generator.batch_size > 0:
    steps_per_epoch += 1

validation_steps = validation_generator.samples // validation_generator.batch_size
if validation_generator.samples % validation_generator.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('Resnet152_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

Found 1493 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Epoch 1/20
94/94 [==============================] - 224s 1s/step - loss: 5.5328 - accuracy: 0.5244 - val_loss: 5.2943 - val_accuracy: 0.4985 - lr: 0.0000e+00
Epoch 2/20
94/94 [==============================] - 16s 173ms/step - loss: 5.0767 - accuracy: 0.7194 - val_loss: 5.2391 - val_accuracy: 0.6625 - lr: 2.0000e-05
Epoch 3/20
94/94 [==============================] - 16s 173ms/step - loss: 4.5361 - accuracy: 0.9129 - val_loss: 5.3301 - val_accuracy: 0.6780 - lr: 4.0000e-05
Epoch 4/20
94/94 [==============================] - 16s 173ms/step - loss: 4.1656 - accuracy: 0.9685 - val_loss: 5.2582 - val_accuracy: 0.7121 - lr: 6.0000e-05
Epoch 5/20
94/94 [==============================] - 16s 173ms/step - loss: 3.8222 - accuracy: 0.9806 - val_loss: 4.8181 - val_accuracy: 0.7121 - lr: 8.0000e-05
Epoch 6/20
94/94 [==============================] - 16s 173ms/step - loss: 3.4980 - accuracy: 0.9806 - val_loss: 4.3501

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Average Validation Accuracy: 0.7316278250593888


## Mobilenet_V3

In [ ]:
from keras.applications.densenet import DenseNet201
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np
from keras.applications import MobileNetV3Large

# MobileNetV3Large 모델 로드 (최상위 층 제외)
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강 없이 이미지 크기만 조정하는 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 데이터셋 로드 및 설정
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = train_generator.samples // train_generator.batch_size
if train_generator.samples % train_generator.batch_size > 0:
    steps_per_epoch += 1

validation_steps = validation_generator.samples // validation_generator.batch_size
if validation_generator.samples % validation_generator.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('Resnet50_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

12683000/12683000 [==============================] - 1s 0us/step
Found 1493 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Epoch 1/20
94/94 [==============================] - 34s 78ms/step - loss: 4.9352 - accuracy: 0.5553 - val_loss: 4.7035 - val_accuracy: 0.6192 - lr: 0.0000e+00
Epoch 2/20
94/94 [==============================] - 5s 57ms/step - loss: 4.7045 - accuracy: 0.6463 - val_loss: 4.6668 - val_accuracy: 0.6192 - lr: 2.0000e-05
Epoch 3/20
94/94 [==============================] - 5s 56ms/step - loss: 4.3788 - accuracy: 0.8051 - val_loss: 4.5802 - val_accuracy: 0.6192 - lr: 4.0000e-05
Epoch 4/20
94/94 [==============================] - 5s 55ms/step - loss: 4.1185 - accuracy: 0.8855 - val_loss: 4.4706 - val_accuracy: 0.6192 - lr: 6.0000e-05
Epoch 5/20
94/94 [==============================] - 5s 57ms/step - loss: 3.8647 - accuracy: 0.9357 - val_loss: 4.3483 - val_accuracy: 0.6192 - lr: 8.0000e-05
Epoch 6/20
94/94 [==============================] - 5s 58ms/s

## Inception_V3

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np

# InceptionV3 모델 로드 (최상위 층 제외)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))  # InceptionV3는 299x299 입력을 사용

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)  # kernel_regularizer는 필요에 따라 추가
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강 없이 이미지 크기만 조정하는 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 데이터셋 로드 및 설정
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(299, 299),  # InceptionV3에 맞게 이미지 크기 조정
    batch_size=16,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(299, 299),  # InceptionV3에 맞게 이미지 크기 조정
    batch_size=16,
    class_mode='binary')

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = train_generator.samples // train_generator.batch_size
if train_generator.samples % train_generator.batch_size > 0:
    steps_per_epoch += 1

validation_steps = validation_generator.samples // validation_generator.batch_size
if validation_generator.samples % validation_generator.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('InceptionV3_drunk_or_sober.h5')

87910968/87910968 [==============================] - 3s 0us/step
Found 1493 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Epoch 1/20
94/94 [==============================] - 44s 132ms/step - loss: 0.9993 - accuracy: 0.4930 - val_loss: 0.7149 - val_accuracy: 0.5170 - lr: 0.0000e+00
Epoch 2/20
94/94 [==============================] - 10s 102ms/step - loss: 0.5559 - accuracy: 0.7294 - val_loss: 0.6375 - val_accuracy: 0.6625 - lr: 2.0000e-05
Epoch 3/20
94/94 [==============================] - 10s 103ms/step - loss: 0.2344 - accuracy: 0.9002 - val_loss: 0.8725 - val_accuracy: 0.6316 - lr: 4.0000e-05
Epoch 4/20
94/94 [==============================] - 10s 103ms/step - loss: 0.0900 - accuracy: 0.9725 - val_loss: 0.7489 - val_accuracy: 0.7214 - lr: 6.0000e-05
Epoch 5/20
94/94 [==============================] - 10s 102ms/step - loss: 0.0473 - accuracy: 0.9859 - val_loss: 0.8508 - val_accuracy: 0.7090 - lr: 8.0000e-05
Epoch 6/20
94/94 [==============================] - 

In [ ]:
# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

Average Validation Accuracy: 0.7220906965872821


## DenseNet201

In [ ]:
from keras.applications.densenet import DenseNet201
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np

# DenseNet201 모델 로드 (최상위 층 제외)
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강 없이 이미지 크기만 조정하는 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 데이터셋 로드 및 설정
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = train_generator.samples // train_generator.batch_size
if train_generator.samples % train_generator.batch_size > 0:
    steps_per_epoch += 1

validation_steps = validation_generator.samples // validation_generator.batch_size
if validation_generator.samples % validation_generator.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('DenseNet201_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

## EffcientB7

In [2]:
from keras.applications.efficientnet import EfficientNetB7
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np

# EfficientNetB0 모델 로드 (최상위 층 제외)
base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강 없이 이미지 크기만 조정하는 ImageDataGenerator 설정
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 데이터셋 로드 및 설정
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = train_generator.samples // train_generator.batch_size
if train_generator.samples % train_generator.batch_size > 0:
    steps_per_epoch += 1

validation_steps = validation_generator.samples // validation_generator.batch_size
if validation_generator.samples % validation_generator.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('EfficientNetB7_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

258076736/258076736 [==============================] - 7s 0us/step
Found 1493 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Epoch 1/20
94/94 [==============================] - 625s 5s/step - loss: 0.9903 - accuracy: 0.5010 - val_loss: 0.7008 - val_accuracy: 0.6192 - lr: 0.0000e+00
Epoch 2/20
94/94 [==============================] - 30s 315ms/step - loss: 0.6465 - accuracy: 0.6899 - val_loss: 0.7112 - val_accuracy: 0.6192 - lr: 2.0000e-05
Epoch 3/20
94/94 [==============================] - 30s 315ms/step - loss: 0.2993 - accuracy: 0.8761 - val_loss: 0.7663 - val_accuracy: 0.6192 - lr: 4.0000e-05
Epoch 4/20
94/94 [==============================] - 30s 315ms/step - loss: 0.1379 - accuracy: 0.9524 - val_loss: 0.7717 - val_accuracy: 0.6099 - lr: 6.0000e-05
Epoch 5/20
94/94 [==============================] - 30s 315ms/step - loss: 0.0928 - accuracy: 0.9618 - val_loss: 0.7744 - val_accuracy: 0.5511 - lr: 8.0000e-05
Epoch 6/20
94/94 [==============================] - 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Average Validation Accuracy: 0.6797213688492775
